# Week 13: Clustering

## Brief Recap:

* Hello, how are you?
* Today: Clustering!
* Next Class (this Friday): an Introductory Guide to Neural Networks
* After Thanksgiving: Presentations!

## Clustering

**Clustering** - the task of identifying similar instances in the data and assigning them to clusters by giving them new class labels. Clusters are subgroupings of similar data.  

Clustering is an unsupervised learning method with many applications:  

* anomoly detection
* customer segmentation (for recommender systems)
* image segmentation
* search engines

## Mixture Densities

Before we dive into Clustering, let's discuss the concept of **Mixture Densities**  

**Mixture Densities** - complicated populations of data sampes that can be expressed as a combination of much simpler subpopulations. e.g. a mixture of gaussians

Let's visualize a mixture of gaussians...

In [ ]:
# bring in some necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
from sklearn.datasets import make_blobs

In [ ]:
n_components = 3
cluster_stds = np.random.uniform(low=2, high=5, size=(n_components,))
X, blob_label = make_blobs(n_samples=300, centers=n_components, 
                      cluster_std = cluster_stds, 
                      random_state=42)
plt.scatter(X[:, 0], X[:, 1], s=50, c = blob_label)
plt.title(f"Example of a mixture of {n_components} distributions")
plt.xlabel("x")
plt.ylabel("y");

### More on Mixture Densities

A mixture density is described mathematically as: $p(\mathbf{x}) = \sum^{k}_{i=1}p(\mathbf{x}|\mathcal{G}_i)P(\mathcal{G}_i)$

$\mathcal{G}_i$ refers to each mixture component (group). We can break this mess of math down as:  

The Mixture Density ($p(\mathbf{x})$) is given by the sum from $1 \rightarrow k$ ($k =$ number of components) of the product of the density of each component ($p(\mathbf{x}|\mathcal{G}_i)$) and the mixture proportion of that component ($P(\mathcal{G}_i)$)

If we assume our system is a mixture of gaussians (as in the python example we plotted),  

* $p(\mathbf{x}|\mathcal{G}_i) \sim \mathcal{N}(\mathbf{\mu}_i, \sum _i))$
* parameters to estimate: weights, means & covariance mats for each cluster

### Classification application of Misture Densities

we can rephrase mixture densities for classification:  

* Groups $\sim$ Classes
* Component Densities $\sim$ Class Densities
* Component Priors $\sim$ Class Priors

We can rewrite: $p(\mathbf{x}) = \sum^{K}_{i=1}p(\mathbf{x}|\mathcal{C}_i)P(\mathcal{C}_i)$

### Supervised Mixture Densities

In the supervised case, we are given the labels along with the observed value. Our task is to learn the parameters that describe the distributions: priors, means & standard deviations 

For a Gaussian Mixture the parameters can be learned with [maximum likelihood](https://courses.cs.duke.edu/spring04/cps196.1/handouts/EM/tomasiEM.pdf)

### Unsupervised Mixture Densities

For the Unsupervised learning problem, we only have the observed value; the labels are unkown.  
What to do?

In [ ]:
plt.scatter(X[:, 0], X[:, 1], s=50)
plt.title(f"Unsupervised mixture of {n_components} distributions")
plt.xlabel("x")
plt.ylabel("y");

### Unsupervised Learning of Class Membership

**Clustering** - the task of identifying similar instances and assigning them to clusters, or groups of similar instances.

To properly describe this mixture system into clusters, we need a different approach to the problem:

1. Estimate Class Labels
2. Estimate the Parameters of the Classes

### Estimating Class Labels using k-Means Custering

k-Means is a simple algorithm for assigning class labels to data points.  
k-Means pseudocoded:  

    Initialize means as 'k' random values
    
    For a maximum number of iterations:  
        For each data point: 
            find the distance between the data point and the k means
            find the closest mean
            assign the datapoint to the closest mean
        Update the k mean values as the mean of all assigned datapoints

#### Naive k-Means

We initialize 'k' means randomly from within a given range.

but there are other options that can behave more optimally  

* **domain knowledge** - can specify wiht domain knowledge if available
* **Random Partition** - randomly assign each observation to one of the 'k', determine the k-means, and iterate from there
* **k-means++** - chose the 1st k randomly from the data points. chose the next ks with a probability proportional to the distance of the neared k that has already been assigned

`sklearn` will use k-means++ by default

#### Determining distance for the k-Means algorithm

* **Euclidean** currently, `sklearn` only support euclidean distance k-means
* **Cosine** cosine of the vector angle
* **Manhattan** sum of distance in all dimensions

#### k-Means Clustering decision

* **hard clustering** - the k-means algorithm assigns class membership for each observation to one and only one class  
    - estimated label = $\left\{ \begin{array}{rcl}
1 & \mbox{if} & \Vert x^t-m_i\Vert = \mbox{min}_j \Vert x^t-m_j \Vert \\ 
0 & \mbox{otherwise}
\end{array}\right.$
    - This can be problematic for observations near class borders.  
* **soft clustering**  algorithms that instead return class membership probability or weights

### Estimating the Clusters' Parameters 

Once we have estimated the class labels, we can estimate the parameters of each class.  
For example, if we assume a mixture of gaussian distributions, it is straight forward to estimate:  

* $P(\mathcal{C}_i)$ - the Class priors 
* $m_i$ - estimate for the mean
* $S_i$ - estimate for the covariance

### Use `sklearn` to perform k-Means

Of course, if you are curious, there are many resources online to guide you through an [implementation of k-means from scratch](https://towardsdatascience.com/k-means-clustering-from-scratch-6a9d19cafc25).

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans( n_clusters = n_components, random_state=0 ).fit( X )
print( kmeans.cluster_centers_ )
kmeans.labels_[0:10]

In [ ]:
# visualize the results of the kMeans fit
f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(12,6))
ax2.scatter(X[:, 0], X[:, 1], s=50, c = kmeans.labels_)
ax2.set_title("k-Means results")
ax1.scatter(X[:, 0], X[:, 1], s=50, c = blob_label)
ax1.set_title(f"Example of a mixture of {n_components} distributions")
plt.show()

### Voronoi Diagram

also called Voronoi Tesselations/Decompositions/Partitions or Dirichlet Tesselations.  
divides the data into Voronoi cells, or Theissian polygons.  

**Voronoi Cell** - each cell has a seed. the cell consists of all data points which are closer to the cell's seed than any other seed. Note that we can use different approaches to define the seed or to determine distance.

In [ ]:
def plot_decision_boundaries(clusterer, X, resolution=1000,
                             show_xlabels=True, show_ylabels=True):
    """
    adapted from Hands-on Machine Learning (Geron 2nd edition)
    """
    mins = X.min(axis=0) - 0.1
    maxs = X.max(axis=0) + 0.1
    xx, yy = np.meshgrid(np.linspace(mins[0], maxs[0], resolution),
                         np.linspace(mins[1], maxs[1], resolution))
    Z = clusterer.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    plt.contourf(Z, extent=(mins[0], maxs[0], mins[1], maxs[1]),
                cmap="Pastel2")
    plt.contour(Z, extent=(mins[0], maxs[0], mins[1], maxs[1]),
                linewidths=1, colors='k')

    if show_xlabels:
        plt.xlabel("$x_1$", fontsize=14)
    else:
        plt.tick_params(labelbottom=False)
    if show_ylabels:
        plt.ylabel("$x_2$", fontsize=14, rotation=0)
    else:
        plt.tick_params(labelleft=False)

In [ ]:
plt.figure(figsize=(12, 9))
plot_decision_boundaries(kmeans, X)
plt.scatter(X[:, 0], X[:, 1], s=50, c = kmeans.labels_)
plt.title(f"Kmeans (k= {n_components} ) with Decision Boundaries")
plt.show()

## Making predictions with new observations

with a trained k-means model, we can easily give class assignments to new observations...

In [ ]:
New_Observations = np.array([[-15,-5],[10,0],[-5,10]])
kmeans.predict(New_Observations)

In [ ]:
# we can also learn the centroids kmeans found
kmeans.cluster_centers_

The previous Voronoi figure illustrated the Class labels determined by k-means.  
K-means is an unsupervised method. However, in this toy data example, we happen to know the ground truth labels for each data point.  

Let's observe the ground truth against the decision boundaries...

In [ ]:
plt.figure(figsize=(12, 9))
plot_decision_boundaries(kmeans, X)
plt.scatter(X[:, 0], X[:, 1], s=50, c = blob_label)
plt.title(f'Kmeans (k={n_components}) with Decision Boundaries')
plt.show()

## Now you try!

Let's pause here.  
Take some time to review the previous code and make some changes. For example:  

* change the `n_components`
* change the cluster_stds low & high parameters

How well does K-means perform when you increase 'k'? or change the spread of the data blobs?

## What is the best 'k'?

Notice how in the previous examples, we fit a k-means model with a 'k' value that was determined by the number of gaussian 'blobs' we added to the toy data set.  

In the real world, when we use k-means, we might not have the luxury of knowing how many 'blobs' we have.  

In fact, 'k' is a hyperparameter that we need to tune and doing so may find a less biased pattern in our data.

## Clustering with the Titanic data set

The Titanic dataset is a popular one. It holds features on passenger information for every passenger listed aboard the Titanic including: age, sex, name, ticket class, fare and even the cabin number to be used for predicting passenger survival (a supervised classification problem).  

Here we will just use a couple of features (passenger age and fare price) to perform a k-means clustering. 

In [ ]:
import pandas as pd
url = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'
titanic = pd.read_csv( url )
titanic.info()

### Light Data Prep

In [ ]:
cluster_df = titanic[['Fare','Age','Survived']].copy()
cluster_df.dropna(axis=0, inplace=True)
cluster_df.head()

### Visualize the Data

In [ ]:
plt.figure(figsize=(12, 9))
plt.scatter(cluster_df['Fare'], cluster_df['Age'], s=50, c = cluster_df['Survived'])
plt.title('Titanic Passenger Data: Fare ~ Age')
plt.show()

### Inertia

**Inertia** - a metric the measures the distance between each instance and its centroid.  

Inertia can help evaluate a k-means model at a given 'k' values.  
However, inertia will necessarily decrease with increasing 'k', as we will see below:

In [ ]:
kmeans_per_k = [KMeans(n_clusters=k, random_state=42).fit(cluster_df[['Fare','Age']])
                for k in range(1, 10)]
inertias = [model.inertia_ for model in kmeans_per_k]
inertias

### What to do? ... the Inertia vs k plot

If we plot inertia for each value of 'k' we can arrive at a good choice of 'k' by finding the 'elbow' in the curve this plot produces.  

**The Elbow** - a lower 'k' would have a dramatically higher inertia, but an higher 'k' would not achieve much.

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(range(1, 10), inertias, "bo-")
plt.xlabel("$k$", fontsize=14)
plt.ylabel("Inertia", fontsize=14)
plt.axis([0, 10, 0, 2000000])
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(range(1, 10), inertias, "bo-")
plt.xlabel("$k$", fontsize=14)
plt.ylabel("Inertia", fontsize=14)
plt.annotate('Elbow',
             xy=(4, inertias[3]),
             xytext=(0.55, 0.55),
             textcoords='figure fraction',
             fontsize=16,
             arrowprops=dict(facecolor='black', shrink=0.1)
            )
plt.axis([0, 10, 0, 2000000])
plt.show()

### Silhouette Diagrams

Did finding the 'elbow' feel uncomfortably subjective?  

Another method is to find the 'k' values with the highest silhouette score. 

 **Silhouette Score** - mean silhouette coefficient for all instances  
**Silhouette Coefficient** - mean distance to the other instances in the same cluster  
$$\mbox{Silhouette Coefficient} = \frac{(b-1)}{\mbox{max}(a,b)}$$
where:  

* a = intra-cluster distance, the mean distance to other instances in the same cluster
* b = mean distance to instances in the nearest cluster

A Silhouette Coefficient ~1 means the instance is well within it's cluster  
"" ~-1 means the instance might have been misassigned

In [ ]:
from sklearn.metrics import silhouette_score

silhouette_scores = [silhouette_score(cluster_df[['Fare','Age']], model.labels_) for model in kmeans_per_k[1:]]

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(range(2, 10), silhouette_scores, "bo-")
plt.xlabel("$k$", fontsize=14)
plt.ylabel("Silhouette score", fontsize=14)
plt.axis([1.8, 9, 0.4, 0.9])
plt.show()

### Silhouette Diagram

The previous figure shows the mean silhouette score for all observations for each cluster.  
However, we can infer a lot more with the Silhouette Diagram:  

[**Silhouette Diagram**](https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html) - plot every instances's silhouette coefficient sorted by cluster membership and value of the coefficient

In [ ]:
# adapted from sklearn documentation
from sklearn.metrics import silhouette_samples
from matplotlib.ticker import FixedLocator, FixedFormatter
import matplotlib.cm as cm

plt.figure(figsize=(12, 9))
X = cluster_df[['Fare','Age']]

for k in (2, 3, 4, 5, 6):
    plt.subplot(2, 3, k - 1)
    
    y_pred = kmeans_per_k[k - 1].labels_
    silhouette_coefficients = silhouette_samples(X, y_pred)

    padding = len(X) // 30
    pos = padding
    ticks = []
    for i in range(k):
        coeffs = silhouette_coefficients[y_pred == i]
        coeffs.sort()

        color = cm.nipy_spectral(float(i) / k)
        plt.fill_betweenx(np.arange(pos, pos + len(coeffs)), 0, coeffs,
                          facecolor=color, edgecolor=color, alpha=0.7)
        ticks.append(pos + len(coeffs) // 2)
        pos += len(coeffs) + padding

    plt.gca().yaxis.set_major_locator(FixedLocator(ticks))
    plt.gca().yaxis.set_major_formatter(FixedFormatter(range(k)))
    if k in (3, 5):
        plt.ylabel("Cluster")
    
    if k in (5, 6):
        plt.gca().set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])
        plt.xlabel("Silhouette Coefficient")
    else:
        plt.tick_params(labelbottom=False)

    plt.axvline(x=silhouette_scores[k - 2], color="red", linestyle="--")
    plt.title("$k={}$".format(k), fontsize=16)
plt.show()

### Interpretting the Silhouette Diagram

The red dashed lines show the mean silhouette score for all observations for each 'k'.  
Compare with the Silhouette Score plot to verify that they are indeed the same values.  

If a cluster does not pass the mean silhouette score, then this is a poorly assigned cluster.  
Our goal is to select a minimal value of 'k' where all cluster cross the mean score threshold.  

Which value of 'k' satisfies this criteria?

### Train a k-means model with k=4

In [ ]:
k = 4
kmeans = KMeans(n_clusters = k)
y_pred = kmeans.fit_predict(X)
X['y_pred'] = y_pred
X.head()

### Visualize the Clustered Data with k=4

In [ ]:
plt.figure(figsize=(12, 9))
plt.scatter(X['Fare'], X['Age'], s=50, c = X['y_pred'])
plt.title(f'Titanic Passenger Data: k={k}')
plt.show()

## Clustering the Iris dataset

Let's try a similar approach with the iris dataset using just the features for the petals

In [ ]:
from sklearn.datasets import load_iris
iris = load_iris()
iris_df = pd.DataFrame(iris.data, columns = iris.feature_names)
subiris = iris_df[['petal length (cm)','petal width (cm)']].copy()
subiris.head()

In [ ]:
plt.figure(figsize=(10, 8))
plt.scatter(subiris['petal length (cm)'], subiris['petal width (cm)'], s=50)
plt.xlabel("petal length (cm)", fontsize=14)
plt.ylabel("petal width (cm)", fontsize=14)
plt.title('Unlabeled Iris')
plt.show()

In [ ]:
# calculate & plot the inertias for various k
kmeans_per_k = [KMeans(n_clusters=k, random_state=42).fit(subiris) for k in range(1, 10)]
inertias = [model.inertia_ for model in kmeans_per_k]
print( inertias )
plt.figure(figsize=(10, 8))
plt.plot(range(1, 10), inertias, "bo-")
plt.xlabel("$k$", fontsize=14)
plt.ylabel("Inertia", fontsize=14)
plt.axis([0, 10, 0, 100])
plt.show()

### Which 'k'?

In [ ]:
k = 3
kmeans = KMeans(n_clusters = k)
y_pred = kmeans.fit_predict(subiris)
subiris['y_pred'] = y_pred

plt.figure(figsize=(12, 9))
plt.scatter(subiris['petal length (cm)'], subiris['petal width (cm)'], s=50, c = subiris['y_pred'])
plt.title(f'Iris Petal Data: k={k}')
plt.show()

## Now you try!
walk through the same code using only the iris 'sepal' features 

## Problems with k-means

* Depending on the initial centroid values, k-means might not reach the same values during training. It is often necessary to run k-means several times to find a solution
* K-means does not behave well if the variances of different clusters are very different
* K-means does not 'like' non-gaussian clusters

## 🥺 The next lecture will be our last lecture class 😢
<img src="https://content.techgig.com/photo/80071467/pros-and-cons-of-python-programming-language-that-every-learner-must-know.jpg?132269" width="100%" style="margin-left:auto; margin-right:auto">